In [1]:
import os
import ee
import geemap
import time
import random
from datetime import datetime, timedelta , date
import ipywidgets as widgets
from IPython.display import display
from tkinter import Tk, filedialog
import traitlets
from ipyfilechooser import FileChooser

In [2]:
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True,filetypes=[('shp','.shp')])

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

In [3]:
# After executing this line of code for the first use, you can get the authentication number linked to Google.
#Map = geemap.Map()
# Authenticate the Google earth engine with google account
ee.Initialize() 
Map = geemap.Map()

In [4]:
# give the shp file
my_button = SelectFilesButton()
my_button # This will display the button in the context of Jupyter Notebook

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [5]:
# give the star date and end date

star = widgets.DatePicker(
    description='Pick a Star Date',
    disabled=False
)
end = widgets.DatePicker(
    description='Pick a End Date',
    disabled=False
)

widgets.HBox([star, end])

In [6]:
# give the bands

band_name = widgets.Dropdown(
options=['Air_2m_T_C_mean','Air_2m_T_C_min','Air_2m_T_C_max','dewpoint_2m_C','RH','mean_2m_air_temperature',
         'minimum_2m_air_temperature','maximum_2m_air_temperature','dewpoint_2m_temperature','total_precipitation',
        'surface_pressure','mean_sea_level_pressure','u_component_of_wind_10m','v_component_of_wind_10m'],
description='Band')

band_name

Dropdown(description='Band', options=('Air_2m_T_C_mean', 'Air_2m_T_C_min', 'Air_2m_T_C_max', 'dewpoint_2m_C', …

In [7]:
# gine the date frequency

frequency =widgets.Dropdown(
    options=['Year','Month'],
    value='Year',
    description='Frequency')

frequency

Dropdown(description='Frequency', options=('Year', 'Month'), value='Year')

In [11]:
# give the output floder and flie name
folder = FileChooser()
display(folder)


FileChooser(path='C:\Users\Yang Hsiu\Desktop\satellite python code\satellite python code\satellite python code…

In [12]:
# give the scale number
scale = widgets.IntSlider(
    value=1000,
    min=500,
    max=10000,
    step=100,
    description='Scale:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

scale

IntSlider(value=1000, continuous_update=False, description='Scale:', max=10000, min=500, step=100)

In [11]:
roi = geemap.shp_to_ee(''.join(my_button.files)).geometry()

In [9]:
def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + timedelta(days=4)  # this will never fail
    return next_month - timedelta(days=next_month.day)

def monthlist(begin,end):
    #begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
    #end = datetime.datetime.strptime(end, "%Y-%m-%d")

    result = []
    while True:
        if begin.month == 12:
            next_month = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_month = begin.replace(month=begin.month+1, day=1)
        if next_month > end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_month(begin).strftime("%Y-%m-%d")])
        begin = next_month
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result



def last_day_of_year(any_day):
    next_year = any_day.replace(year=any_day.year+1, month=12, day = 31)  # this will never fail
    return next_year

def yearlist(begin,end):
    #begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
    #end = datetime.datetime.strptime(end, "%Y-%m-%d")

    result = []
    while True:
        if begin.year < end.year:
            next_year = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_year = end
        if next_year == end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_year(begin).strftime("%Y-%m-%d")])
        begin = next_year
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result

In [13]:
def getC_air_mean(image):
    
    Air_2m_T_C_mean = image.expression(
      'T - 273.15',{
      'T' : image.select('mean_2m_air_temperature')}).rename('Air_2m_T_C_mean');
    
    return image.addBands(Air_2m_T_C_mean)

def getC_air_min(image):
    
    Air_2m_T_C_min = image.expression(
      'T - 273.15',{
      'T' : image.select('minimum_2m_air_temperature')}).rename('Air_2m_T_C_min');
    
    return image.addBands(Air_2m_T_C_min)

def getC_air_max(image):
    
    Air_2m_T_C_max = image.expression(
      'T - 273.15',{
      'T' : image.select('maximum_2m_air_temperature')}).rename('Air_2m_T_C_max');
    
    return image.addBands(Air_2m_T_C_max)


def getC_dewpoint(image):
    
    dewpoint_2m_temperature_C = image.expression(
      'T - 273.15',{
      'T' : image.select('dewpoint_2m_temperature')}).rename('dewpoint_2m_C');
    
    return image.addBands(dewpoint_2m_temperature_C)

def getRH(image):
    
    RH = image.expression(
      '100 * (exp((17.625 * Td)/(243.04 + Td))/exp((17.625*T)/(243.04 + T)))',{
      'Td' : image.select('dewpoint_2m_temperature').add(-273.15),
      'T' : image.select('mean_2m_air_temperature').add(-273.15)}).rename('RH');
    
    return image.addBands(RH)

In [14]:
def geotif(time_list):
    for i in range(0,len(time_list)):
        star_time = time.time()


        dataset = ee.ImageCollection("ECMWF/ERA5/DAILY") \
                             .filterDate(time_list[i][0],datetime.strptime(time_list[i][1],"%Y-%m-%d")+ timedelta(days=1)) \
                             .filterBounds(roi) \
                             .map(getC_air_mean)\
                             .map(getC_air_max)\
                             .map(getC_air_min)\
                             .map(getC_dewpoint)\
                             .map(getRH)\
                             .select(band_name.value)\
                             .mean()

        image = dataset.clip(roi).unmask()
        filename = os.path.join(os.path.expanduser(folder.selected) + '_{}.tif'.format(time_list[i]))
        geemap.ee_export_image(
            image, filename=filename, scale=scale.value, region=roi, file_per_band=False
                                )

In [15]:
if 'Year' in frequency.value.strip():
    time_list = yearlist(star.value, end.value)
    geotif(time_list)
else:
    pass
if "Month" in frequency.value.strip():
    time_list =monthlist(star.value, end.value)
    geotif(time_list)
else:
    pass

Generating URL ...
Please wait ...
Data downloaded to D:\12312313_['2018-02-06', '2018-12-20'].tif
